In [1]:
import math

In [2]:
# same lenght
def deltaVectorsNorm(x, y):
    res = 0.0
    for i in range (0, len(x)):
        res += (y[i] - x[i]) ** 2
    return math.sqrt(res)

In [3]:
# Jakobi: method = 0
# Seidel: method = 1 (determinant(B) should be less 1)
def templateJakobiAndSeidel(A, b, epsilon, method):
    iterations = 0
    B = []
    d = []
    xn = []
    
    for i in range (0, len(A)):
        d.append(b[i] / A[i][i])
        xn.append(d[i])
        B.append([])
        for j in range (0, len(A[i])):
            B[i].append(-A[i][j] / A[i][i])
        B[i][i] = 0.0
    
    # detB = determinant(B)
    # eps = (1 - detB) * epsilon / detB
    eps = epsilon
    x = xn
    
    while ((deltaVectorsNorm(x, xn) < eps) and iterations < 50):
        x = xn
        for i in range (0, len(B)):
            xn[i] = 0.0
            for j in range (0, len(B[i])):
                xn[i] += B[i][j] * (xn[j] if (j < i and method == 1) else x[j])
            xn[i] += d[i]
        iterations = iterations + 1
            
    
    return (xn, iterations)

In [4]:
def jakobi(A, b, epsilon):
    return templateJakobiAndSeidel(A, b, epsilon, 0) 

In [5]:
def seidel(A, b, epsilon):
    return templateJakobiAndSeidel(A, b, epsilon, 1) 

In [6]:
def test_jakobi():
    A = [[10,1,-1],[1,10,-1],[-1,1,10]]
    b = [11,10,10]
    print ("Jakobi")
    (ans, it) = jakobi(A, b, 0.001)
    print (ans)
    print (it)

test_jakobi()

Jakobi
[1.1020202020202021, 0.9909090909090909, 1.011111111111111]
50


In [7]:
def test_seidel():
    A = [[10,1,-1],[1,10,-1],[-1,1,10]]
    b = [11,10,10]
    print ("Seidel")
    (ans, it) = seidel(A, b, 0.001)
    print (ans)
    print (it)

test_seidel()

Seidel
[1.1020202020202021, 0.9909090909090909, 1.011111111111111]
50
